### Reflection pattern

Here we pass back the response with another request to critique the response.

This pattern can be useful in RAG workflows where we ask the LLM to assess the previous response.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-Dg


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt.",
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(
        messages=generation_chat_history, model=MODEL
    )
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Sure! Below is a simple yet comprehensive Python implementation that demonstrates how to make a GET request to an API using the `requests` library. This example illustrates error handling, setting headers, and processing the response.

```python
import requests

def fetch_data_from_api(url, headers=None, params=None):
    """
    Fetch data from the specified API endpoint.

    :param url: str - The endpoint URL of the API.
    :param headers: dict - Optional. Custom headers to send with the request.
    :param params: dict - Optional. Query parameters to include in the request.
    :return: dict - The JSON response from the API if successful, otherwise None.
    """
    try:
        # Make a GET request to the API
        response = requests.get(url, headers=headers, params=params)

        # Raise an exception for HTTP errors (4xx and 5xx)
        response.raise_for_status()

        # Process the JSON response
        return response.json()

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # e.g., 404 Not Found
    except requests.exceptions.RequestException as req_err:
        print(f"Error occurred: {req_err}")  # Other errors (e.g., connection error, timeout)
    except ValueError as json_err:
        print(f"Error parsing JSON: {json_err}")  # When response content is not valid JSON

    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    custom_headers = {
        "Authorization": "Bearer YOUR_ACCESS_TOKEN",
        "Accept": "application/json"
    }
    query_params = {
        "param1": "value1",
        "param2": "value2"
    }

    data = fetch_data_from_api(api_url, headers=custom_headers, params=query_params)
    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

### Key Features:
- **Error Handling:** The code includes error handling using try-except blocks for HTTP errors, request errors, and JSON parsing errors.
- **Headers and Parameters:** It demonstrates how to set custom headers and query parameters.
- **Modular Function:** The API request logic is encapsulated within a function, making it reusable and easier to test.

Feel free to modify the `api_url`, `custom_headers`, or `query_params` according to your needs!

## Reflection Step


This is equivalent to asking a followup question in say ChatGPT


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(
        messages=reflection_chat_history, model=MODEL
    )
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your implementation of the API request using the `requests` library is well-structured and covers essential aspects such as error handling, customization through headers and parameters, and a modular approach that promotes reusability. Below are some critiques and recommendations to enhance your code further:

### Critique

1. **Verbose Output Handling**:
   - You print error messages directly to the console, which might be okay for debugging but isn't the best practice for production code. Instead, consider logging the errors using the `logging` module. This provides more control over output and can help in tracking issues in production environments.

2. **Returning More Context on Errors**:
   - While you do handle exceptions, you might want to return or log more context about the error. Including the URL and parameters that were used could help in diagnosing issues when something goes wrong.

3. **Docstring Enhancements**:
   - The docstring is informative but could be improved by specifying the expected response structure more clearly. An example of the expected output would also help users understand what to expect from the function.

4. **Type Hinting**:
   - Consider using type hints for the parameters and the return type in your function definition. This improves code readability and helps users understand what types of arguments are expected.

5. **Parameter Validation**:
   - You may want to include some basic validation for the `url`, `headers`, and `params` to ensure that they conform to expected formats before making the request.

6. **Handling Non-JSON Responses**:
   - If the API returns a non-JSON response (e.g., plain text or HTML), it would be more robust to handle that gracefully instead of assuming it's always JSON.

### Recommendations

Here is a modified version of your function that takes into account some of these critiques and recommendations:

```python
import requests
import logging
from typing import Optional, Dict, Any

# Configure logging
logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url: str, headers: Optional[Dict[str, str]] = None, params: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    """
    Fetch data from the specified API endpoint.

    :param url: str - The endpoint URL of the API.
    :param headers: dict - Optional. Custom headers to send with the request.
    :param params: dict - Optional. Query parameters to include in the request.
    :return: dict - The JSON response from the API if successful, otherwise None.
    """
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        try:
            return response.json()
        except ValueError:
            logging.error("Error parsing JSON - response text: %s", response.text)
            return None

    except requests.exceptions.HTTPError as http_err:
        logging.error("HTTP error occurred for URL %s: %s", url, http_err)
    except requests.exceptions.RequestException as req_err:
        logging.error("Request error for URL %s: %s", url, req_err)

    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    custom_headers = {
        "Authorization": "Bearer YOUR_ACCESS_TOKEN",
        "Accept": "application/json"
    }
    query_params = {
        "param1": "value1",
        "param2": "value2"
    }

    data = fetch_data_from_api(api_url, headers=custom_headers, params=query_params)
    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

### Summary

Your initial code is a solid foundation for fetching data from an API. By incorporating structured logging, type hints, enhanced error handling, and input validation, you can improve the maintainability and robustness of your code. This will also make it easier for others to understand and use your function in various contexts. Keep up the great work!

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(
        messages=generation_chat_history, model=MODEL
    )
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for your insightful critique and recommendations! I've revised the implementation based on your feedback, adding structured logging, improved error handling with more context, enhanced docstrings, and type hints. Here is the updated version of the Python code for requesting an API using the `requests` library:

```python
import requests
import logging
from typing import Optional, Dict, Any

# Configure logging
logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url: str, headers: Optional[Dict[str, str]] = None, params: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    """
    Fetch data from the specified API endpoint.

    :param url: str - The endpoint URL of the API.
    :param headers: dict - Optional. Custom headers to send with the request.
    :param params: dict - Optional. Query parameters to include in the request.
    :return: dict - The JSON response from the API if successful, otherwise None.
    
    Example response:
    {
        "key1": "value1",
        "key2": "value2"
    }
    """
    if not isinstance(url, str) or not url.startswith(('http://', 'https://')):
        logging.error("Invalid URL provided: %s", url)
        return None

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        try:
            return response.json()
        except ValueError:
            logging.error("Error parsing JSON - response text: %s", response.text)
            return None

    except requests.exceptions.HTTPError as http_err:
        logging.error("HTTP error occurred for URL %s: %s", url, http_err)
    except requests.exceptions.RequestException as req_err:
        logging.error("Request error for URL %s: %s", url, req_err)

    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    custom_headers = {
        "Authorization": "Bearer YOUR_ACCESS_TOKEN",
        "Accept": "application/json"
    }
    query_params = {
        "param1": "value1",
        "param2": "value2"
    }

    data = fetch_data_from_api(api_url, headers=custom_headers, params=query_params)
    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

### Key Improvements:
1. **Logging Configuration**: I set up logging to capture error messages, providing more control over output and facilitating troubleshooting.
2. **Parameter Validation**: Added basic validation for the URL parameter to ensure it's a proper HTTP or HTTPS URL.
3. **Enhanced Error Handling**: The exceptions now log the URL and the specific errors that occur, aiding in debugging.
4. **Docstring Example**: Included an example of what the response structure might look like, enhancing clarity for potential users.

This revised version aims to be more robust, maintainable, and user-friendly. Thank you again for your valuable feedback, and I'm here to help if you have any more questions or need further enhancements!

<br>

## We can of course make this a Class...
